In [1]:
! pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.8 MB/s eta 0:00:00


In [2]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
import torch
from datasets import load_dataset
from sacrebleu import corpus_bleu
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq
from torch.utils.data import Dataset

In [3]:
model_name="facebook/m2m100_1.2B"

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = M2M100Tokenizer.from_pretrained(model_name)
model = M2M100ForConditionalGeneration.from_pretrained(model_name).to(device)
tokenizer.src_lang = "ru"

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/909 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

In [5]:
def translate(russian_texts):
  inputs = tokenizer(russian_texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

  generated_tokens = model.generate(
      **inputs,
      forced_bos_token_id=tokenizer.et_lang_id("en"),
      max_length=512)


  translations = tokenizer.batch_decode(
      generated_tokens,
      skip_special_tokens=True)

  return translations


In [6]:
dataset = load_dataset("dsmchr/Parallel_web_fantasy_ru_en")

README.md: 0.00B [00:00, ?B/s]

filtered_fantasy_lit_ru_en.tsv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/25787 [00:00<?, ? examples/s]

In [7]:
split_dataset = dataset['train'].train_test_split(
            test_size=0.2,
            seed=42
        )

In [8]:
train_data = split_dataset['train']
test_data = split_dataset['test']

In [9]:
def trim_dataset(train_size, test_size):

    ru_train_trimmed = train_data['russian_text'][:train_size]
    en_train_trimmed = train_data['english_text'][:train_size]


    ru_test_trimmed = test_data['russian_text'][:test_size]
    en_test_trimmed = test_data['english_text'][:test_size]

    return ru_train_trimmed, en_train_trimmed, ru_test_trimmed, en_test_trimmed




In [10]:
train_samples = 100
test_samples = 1000

ru_train, en_train, ru_test, en_test = trim_dataset(train_samples, test_samples)

In [11]:
def evaluate_bleu(model, tokenizer, source_sentences, reference_sentences, batch_size=8):
    hypothesis = []


    for i in range(0, len(source_sentences), batch_size):
        batch_source = source_sentences[i:i + batch_size]


        inputs = tokenizer(
            batch_source,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128
        ).to(device)

        generated_tokens = model.generate(
            **inputs,
            forced_bos_token_id=tokenizer.get_lang_id("en"),
            max_length=128,
            num_beams=5
        )

        batch_translations = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        hypothesis.extend(batch_translations)

        if i % 50 == 0:
            print(f"Processed {i}/{len(source_sentences)} sentences")

    references = [reference_sentences]

    bleu_score = corpus_bleu(hypothesis, references)
    return bleu_score.score, hypothesis

In [12]:
bleu_score, translations = evaluate_bleu(model, tokenizer, ru_test, en_test)

Processed 0/1000 sentences
Processed 200/1000 sentences
Processed 400/1000 sentences
Processed 600/1000 sentences
Processed 800/1000 sentences


In [13]:
print(f"\nBLEU score: {bleu_score:.2f}")


for i in range(10):
  print(f"RU: {ru_test[i]}")
  print(f"EN Reference: {en_test[i]}")
  print(f"EN Hypothesis: {translations[i]}\n")


BLEU score: 22.04
RU: Благо он заметно успокоился, поняв, что Маркус не искал проблем.
EN Reference: To his credit, he seemed to calm down and gathered his bearing quickly after realizing Marcus wasn’t looking for trouble.
EN Hypothesis: Fortunately, he remarkably calmed down, realizing that Marcus was not looking for problems.

RU: Быстро проверив рощу, Дельта замерла.
EN Reference: Which a quick check on the grove, Delta froze.
EN Hypothesis: Upon checking the grove, the delta froze.

RU: Невероятные новости!
EN Reference: Very good news indeed! ”
EN Hypothesis: Incredible news !

RU: Я продолжаю двигаться все дальше и дальше.
EN Reference: I keepmovingmore and more.
EN Hypothesis: I continue to move further and further.

RU: С его помощью я опознаю троих мужчин и двух женщин, чье исчезновение вызвало бы шумиху.
EN Reference: With his help, I identify three men and two women whose disappearance would cause an uproar.
EN Hypothesis: With his help, I identify three men and two women w